<a href="https://colab.research.google.com/github/andzagz/clusterizacao-cnpj/blob/main/Movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [ ]:
import requests
import pandas as pd

# Function to retrieve the film's budget from its Detail URL
def get_budget_from_detail_url(detail_url):
    try:
        # Perform a GET request to the film's Detail URL
        detail_response = requests.get(detail_url)

        # Check if the request was successful
        if detail_response.status_code == 200:
            detail_data = detail_response.json()  # Convert the response to JSON
            budget = detail_data.get('Budget', 'N/A')  # Extract the 'Budget' value
            return budget
        else:
            return 'N/A'  # Return 'N/A' if the request failed
    except Exception as e:
        # Return an error message in case of exceptions (e.g., network issues)
        return 'Error'

# Main function to retrieve film data from the provided URL
def get_films_data(url):
    film_data = []  # Initialize an empty list to store film data

    # Perform a GET request to the main URL to retrieve the film list
    response = requests.get(url)

    # Verify if the request was successful (HTTP status 200)
    if response.status_code == 200:
        data = response.json()  # Convert the response to JSON format

        # Iterate over the results (grouped by year)
        for result in data['results']:
            year = result.get('year', 'N/A')  # Extract the 'year' from the data

            # Iterate over the films in the current result set
            for film in result['films']:
                film_title = film.get('Film', 'N/A')  # Extract the film's title
                wiki_url = film.get('Wiki URL', 'N/A')  # Extract the Wikipedia URL
                winner = film.get('Winner', 'N/A')  # Extract the 'Winner' status
                detail_url = film.get('Detail URL', 'N/A')  # Extract the Detail URL

                # Retrieve the film's budget using the Detail URL
                budget = get_budget_from_detail_url(detail_url)

                # Append the extracted film details to the list
                film_data.append({
                    'Film': film_title,
                    'Year': year,
                    'Wiki URL': wiki_url,
                    'Winner': winner,
                    'Budget': budget
                })
    else:
        # Print a message if the request to the main URL failed
        print(f"Request failed. Status code: {response.status_code}")

    # Return the collected film data as a DataFrame
    return pd.DataFrame(film_data)

# Main URL where the film data is located
url = 'http://oscars.yipitdata.com/'

# Fetch the film data and store it in a DataFrame
df_films = get_films_data(url)

# Export the DataFrame to an Excel file
df_films.to_excel('film_details.xlsx', index=False)

print("Data exported to 'film_details.xlsx'")


Data exported to 'film_details.xlsx'


In [ ]:
from google.colab import files
files.download('filmes_detalhes.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load the DataFrame from the previously created Excel file
df_films = pd.read_excel('film_details.xlsx')

# Create a new column 'Budget1' by copying data from the 'Budget' column
# Replace invalid budget values ('N/A' and 'Erro') with 0 and fill any missing values with 0
df_films['Budget1'] = df_films['Budget'].replace({'N/A': 0, 'Erro': 0}).fillna(0)

# Export the updated DataFrame to a new Excel file
df_films.to_excel('updated_film_details.xlsx', index=False)

print("Data updated and exported to 'updated_film_details.xlsx'")


Data updated and exported to 'updated_film_details.xlsx'


In [ ]:
from google.colab import files
files.download('filmes_detalhes_atualizados.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import re

# Load the DataFrame from the previously updated Excel file
df_films = pd.read_excel('updated_film_details.xlsx')

# Dictionary of exchange rates (example, fictional values)
exchange_rates = {
    '£': 1.37,  # British Pound to US Dollar
    '₤': 1.37,  # British Pound to US Dollar
    '€': 1.10,  # Euro to US Dollar
    'US$': 1.0,  # US Dollar to US Dollar
    '$': 1.0    # US Dollar to US Dollar
}

def convert_million_to_number(value_str):
    """Convert values containing 'million' into a full numeric format."""
    try:
        return f"{float(value_str.replace(',', '')) * 1_000_000:,.0f}".replace(",", ".")
    except ValueError:
        return 'N/A'

def extract_budget(budget_str):
    """Extract and convert the budget from the given string format."""
    if not isinstance(budget_str, str):  # Check if budget_str is a string
        return 'N/A'

    # Remove brackets and any content within them
    budget_str = re.sub(r'\[\s*\d+\s*\]', '', budget_str)

    # 1. Handle cases with "million" in ranges, such as "$8–8.5 million"
    range_million_matches = re.findall(r'\$([\d,.]+)\s*[–-]\s*\$?([\d,.]+)\s*million', budget_str, re.IGNORECASE)
    if range_million_matches:
        # Get the minimum value from the range and convert to millions
        min_value = min(float(match.replace(',', '')) for match in range_million_matches[0])
        return f"${min_value * 1_000_000:,.0f}".replace(",", ".")

    # 2. Handle cases with "million" in different currencies
    million_matches = re.findall(r'([£₤€$US\$])\s*([\d,.]+)\s*million(?:\s*\[\s*\d+\s*\])?', budget_str, re.IGNORECASE)
    if million_matches:
        # Convert to full numeric format
        return f"{million_matches[0][0]}{convert_million_to_number(million_matches[0][1])}"

    # 3. If there are ranges, get the minimum value (without "million")
    range_matches = re.findall(r'\$([\d,]+(?:\.\d+)?)\s*–\s*\$([\d,]+(?:\.\d+)?)', budget_str, re.IGNORECASE)
    if range_matches:
        return f"${min(float(match.replace(',', '')) for match in range_matches[0]):,.0f}"

    # 4. Extract direct dollar values, with or without references
    dollar_matches = re.findall(r'\$\s*([\d,]+(?:\.\d+)?)(?:\s*million)?(?:\s*\[\s*\d+\s*\])?', budget_str)
    if dollar_matches and 'million' not in budget_str.lower():
        return f"${dollar_matches[0]}"

    # 5. Check for other currency identifiers and direct values
    other_currency_matches = re.findall(r'([£₤€]|US\$)\s*([\d,]+(?:\.\d+)?)(?:\s*million)?(?:\s*\[\s*\d+\s*\])?', budget_str)
    if other_currency_matches and 'million' not in budget_str.lower():
        return f"{other_currency_matches[0][0]}{other_currency_matches[0][1]}"

    # 6. Handle cases with "or" and multiple options
    or_matches = re.findall(r'\$(\d+(?:,\d{3})*(?:\.\d+)?)\s*or\s*([£₤€])\s*(\d+(?:,\d{3})*(?:\.\d+)?)', budget_str)
    if or_matches:
        return f"${or_matches[0][0]}"

    # 7. If no value is found, return N/A
    return 'N/A'

# Create the new 'Budget2' column by applying the extraction function
df_films['Budget2'] = df_films['Budget1'].apply(extract_budget)

# Function to extract the currency symbol and numeric value from the budget
def extract_currency_and_value(budget_str):
    if not isinstance(budget_str, str) or budget_str == 'N/A':
        return 'N/A', 'N/A'

    # 1. Extract the currency symbol (first character)
    currency = budget_str[0]

    # 2. Extract the numeric value (remove symbol and formatting)
    value_str = re.sub(r'[^\d]', '', budget_str)  # Remove anything that is not a number
    return currency, value_str

# Apply the function to create 'Currency' and 'Value' columns
df_films[['Currency', 'Value']] = df_films['Budget2'].apply(lambda x: pd.Series(extract_currency_and_value(x)))

# Function to convert values to USD based on the currency
def convert_to_usd(currency, value):
    if currency not in exchange_rates:
        return 0  # Return 0 if the currency is not in the dictionary
    value_numeric = float(value) if value != 'N/A' else 0
    return value_numeric * exchange_rates[currency]

# Create the 'USD Value' column by applying the conversion function
df_films['USD Value'] = df_films.apply(lambda row: convert_to_usd(row['Currency'], row['Value']), axis=1)

# Replace missing values, 'N/A', or NaN with 0 across all columns
df_films.fillna(0, inplace=True)
df_films.replace('N/A', 0, inplace=True)

# Update the 'Year' column to contain the first 4 characters only (for formatting)
df_films['Year'] = df_films['Year'].str.slice(0, 4)

# Export the updated DataFrame to a new Excel file
df_films.to_excel('film_details_with_budget2_and_new_columns.xlsx', index=False)

print("Data updated and exported to 'film_details_with_budget2_and_new_columns.xlsx'")


Data updated and exported to 'film_details_with_budget2_and_new_columns.xlsx'


In [ ]:
from google.colab import files
files.download('filmes_detalhes_com_budget2_e_novas_colunas.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Validation Step
# 1) Check and replace missing values with 0
if df_films.isnull().values.any():
    print("Missing values found. Replacing with 0.")
    df_films.fillna(0, inplace=True)

# 2) Check and remove duplicate values in the "Film" column
duplicates = df_films.duplicated(subset=['Film'])
if duplicates.any():
    print("Duplicate values found in the 'Film' column. Removing duplicates.")
    df_films.drop_duplicates(subset=['Film'], inplace=True)

# Export the updated DataFrame to a new Excel file
df_films.to_excel('validated_film_details.xlsx', index=False)

print("Validation completed and data exported to 'validated_film_details.xlsx'")


Duplicate values found in the 'Film' column. Removing duplicates.
Validation completed and data exported to 'validated_film_details.xlsx'


In [ ]:
# Resetting the format of columns
df_films['Film'] = df_films['Film'].astype(str)  # Convert to string
df_films['Year'] = pd.to_datetime(df_films['Year'].astype(str), format='%Y').dt.year  # Convert to datetime (year)
df_films['Wiki URL'] = df_films['Wiki URL'].astype(str)  # Convert to string
df_films['Winner'] = df_films['Winner'].map({'Y': True, 'N': False}).fillna(False)  # Convert to boolean (True/False)
df_films['USD Value'] = df_films['USD Value'].astype(int)  # Convert to integer
df_films = df_films.drop(columns=['Budget1', 'Budget2', 'Currency', 'Value'])  # Drop unnecessary columns
# Export the new DataFrame to a CSV file
df_films.to_csv('final_film_details.csv', index=False)

print("New DataFrame exported to 'final_film_details.csv'")


New DataFrame exported to 'final_film_details.csv'


<ipython-input-22-21299e0cf33a>:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films['Winner'] = df_films['Winner'].map({'Y': True, 'N': False}).fillna(False)  # Convert to boolean (True/False)


In [ ]:
from google.colab import files
files.download('final_film_details.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# Create the content for README.md
readme_content = """
# ETL Project: Movie Data Analysis

This project aims to perform an ETL (Extraction, Transformation, and Load) process to collect, process, and analyze data related to movies.

## Project Structure

The project consists of a Python script utilizing the `pandas` and `requests` libraries to perform the following stages:

1. **Extraction**: Data collection from an API endpoint that provides information on movies.
2. **Transformation**: Processing the extracted data.
   2.1 The core logic relies on regular expressions (REGEX) to transform the data from the available endpoints.
       Key transformations include cleaning the "Budget" and "Year" fields according to their specific characteristics.
       2.1.1 The "Budget" field presents various challenges, such as differing currency symbols, units, missing values, ranges, and formats.
       2.1.1.1 The first function, `convert_million_to_number`, is designed to identify and standardize string values representing different units into a common number or unit.
       2.1.1.2 The second function, `extract_budget`, identifies the different currencies and ranges, applying a uniform pattern.
       2.1.1.3 The third function, `convert_to_usd`, converts all identified currencies into USD.
3. **Load**: Exporting the processed data to a CSV file.

## Usage

1. Clone this repository or download the Python script.
2. Run the script in a Python environment.

## Features

- Collects movie budget data from an API.
- Processes and cleans the collected data.

### Example Usage:
#### Functions Created:
- `extract_and_transform_budget(budget_df)`

**Purpose**: Extract and transform the budget values in the "Budget1" column and create the columns "Budget2," "Currency," "Value," and "USD Value."
**Example Application**:
```python
extract_and_transform_budget(df)
"""

# Salvar o conteúdo em um arquivo README.md
with open('README.md', 'w') as f:
    f.write(readme_content)

# Download do arquivo README.md
files.download('README.md')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>